# Chemical potential of a van der Waals real gas


**Code:** #117-000

**File:** `apps/van_der_waals/chemical_potential.ipynb`

-----------

In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.widget-label { display: contents !important; }</style>"))
display(HTML("<style>.slider-container { margin: 12px !important; }</style>"))
display(HTML("<style>.jupyter-widgets { overflow: auto !important; }</style>"))

In [2]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

## Physical functions

In [3]:
def get_relative_isotherms(v_range, T_range):
    """This function calculates the theoretical p(v, T) plane 
        (in reduced coordinates) according to van der Waals
        equation of state from a given range of volumes
        and tenperatures.

    Args:
        v_range: An array containing the values of v
        (in reduced coordinates)for which the isotherms must be 
        calculated.\n
        T_range: An array containing the values of v
        (in reduced coordinates)for which the isotherms must be 
        calculated.\n

        
    Returns:
        isotherms: A list consisted of numpy arrays containing the
        pressures of each isotherm.
    """

    isotherms = []

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms.append(p_R)

    return isotherms

    return isotherms

In [4]:
def experimental_isotherms(p_range, v_range, T_range, fixed_p, fixed_T):
    """This function calculates the experimental p(v, T) plane 
        (in reduced coordinates) according to van der Waals
        equation of state for a given range of volumes
        and tenperatures or for a given range of volumes
        and pressures.

    Args:
        p_range: An array containing the values of p
        (in reduced coordinates)for which the isotherms must be 
        calculated. Only used if fixed_p == True.\n
        v_range: An array containing the values of v
        (in reduced coordinates)for which the isotherms must be 
        calculated.\n
        T_range: An array containing the values of v
        (in reduced coordinates)for which the isotherms must be 
        calculated. Only used if fixed_T == True.\n
        fixed_p: Boolean variable which represents if the isotherms 
        must be calculated for a given pressures.\n
        fixed_T: Boolean variable which represents if the isotherms 
        must be calculated for a given pressures.\n

        
    Returns:
        expe_data: A list consisted of numpy arrays containing the
        pressures of each theoretical isotherm.\n
        theo_data: A list consisted of numpy arrays containing the
        pressures of each theoretical isotherm.\n
        v_limits: A list consisted of arrays of the volume limits of
        the phase-transition of each subcritical isotherm.\n
        p_limits: A list consisted of arrays of the pressure limits of
        the phase-transition of each subcritical isotherm.\n
        tenperatures: A list consisted of the tenperatures of the
        isotherms.\n
    """
    
    if fixed_T:
    
        theo_data = get_relative_isotherms(v_range, T_range)
        expe_data = []

        v_limits = []
        p_limits = []

        p_range = np.linspace(0.001, 1.0, num=10000)
        pressures, v_isobaric_limits = find_real_fixed_T(p_range, T_range)

        for i in range(len(theo_data)):

            p_expe = []

            if i < len(v_isobaric_limits):

                    v_lim = v_isobaric_limits[i]

                    if len(v_lim) > 1: #check if there is only one point 
                        for j in range(len(v_range)):

                            if v_range[j] > v_lim[0] and v_range[j] < v_lim[1]:
                                p_expe.append(pressures[i])

                            else:
                                p_expe.append(theo_data[i][j])

                        v_limits = np.append(v_limits, [v_lim[0], v_lim[1]])
                        p_limits = np.append(p_limits, [pressures[i], pressures[i]])

                    else:
                        p_expe = theo_data[i]
                        v_limits = np.append(v_limits, [1.0])
                        p_limits = np.append(p_limits, [1.0])

            else:

                p_expe = theo_data[i]

            expe_data.append(p_expe)      

        tenperatures = T_range
        
        return expe_data, theo_data, p_limits, v_limits, tenperatures
    
    elif fixed_p:
        
        tenperatures, v_isobaric_limits = find_real_fixed_p(p_range, T_range)

        theo_data = get_relative_isotherms(v_range, tenperatures)
        expe_data = []

        v_limits = []
        p_limits = []
        
        for i in range(len(theo_data)):

            p_expe = []

            if i < len(v_isobaric_limits):

                    v_lim = v_isobaric_limits[i]

                    if len(v_lim) > 1: #check if there is only one point 
                        
                        for j in range(len(v_range)):

                            if v_range[j] > v_lim[0] and v_range[j] < v_lim[1]:
                                p_expe.append(p_range[i])

                            else:
                                p_expe.append(theo_data[i][j])

                        v_limits = np.append(v_limits, [v_lim[0], v_lim[1]])
                        p_limits = np.append(p_limits, [p_range[i], p_range[i]])

                    else:
                        p_expe = theo_data[i]
                        v_limits = np.append(v_limits, [1.0])
                        p_limits = np.append(p_limits, [1.0])

            else:

                p_expe = theo_data[i]

            expe_data.append(p_expe)      

        return expe_data, theo_data, p_limits, v_limits, tenperatures

In [5]:
def get_roots(p, T):
    """This function finds the intersection between an isobaric curve
       and Van der Waals equation of state for a given T.\n
       Values of v with no physical meaning are dismissed
       (v < 0 or complex).

    Args:
        p: Pressure of the isobaric curve.\n
        T: Tenperature of the isotherm.\n

        
    Returns:
        roots_in_range: A sorted list of the volumes in which the
        isobaric curve intersects the isotherm.\n
    """
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        
        # A third degree polynomial has 3 complex roots,
        # but we are only interested in the ones which are
        # purely real.
        
        if np.isreal(root):
            
            root = np.real(root)
            
            if root > 0:
                
                roots_in_range.append(root)

    roots_in_range.sort()
    
    return roots_in_range

In [6]:
def p_indefinite_integral(p_0, v_0, T):
    """This function calculates the indefinite integral between
    a van der Waals isotherm and a isobaric line.
    
    Args:
        p0: Isobaric line's pressure.\n
        v0: Value of the volume.\n
        T: Value of the tenperature.\n

    Returns:
        integral: Value of the indefinite integral between a 
        van der Waals isotherm at T and a isobaric line of p0 at a
        volume v0.\n
    """
    
    integral = 8.0/3.0 * T *np.log(v_0 - 1.0/3.0) + 3.0/v_0 - p_0*v_0
    
    return integral

In [7]:
def definite_integral(p_0, v_range, T):
    """This function 'p_indefinite_integral' function to calculate
    the definite integral between a van der Waals isotherm and a
    isobaric line.
    
    Args:
        p0: Isobaric line's pressure.\n
        v_range: Tuple or list consisted of volume limits.\n
        T: Value of the tenperature.\n

    Returns:
        integral: Value of the definite integral between a 
        van der Waals isotherm at T and a isobaric line of p0 in a
        volume range v_range.\n
    """
    
    v_0, v_1 = v_range[0], v_range[1]
    
    integral = p_indefinite_integral(p_0, v_1, T) - p_indefinite_integral(p_0, v_0, T)
    
    return integral

In [8]:
def find_real_fixed_T(p_values, T_values):
    """This function uses Maxwell's construction to find the
       pressures in which phase transition happens given some
       fixed tenperatures.\n

    Args:
        p_values: List of pressures in which the real isotherm is
        searched.\n
        T_values: List of tenperatures of the isotherms.\n

        
    Returns:
        pressures: List of pressures in which phase transition
        happens.\n
        v_range: Volume limits of phase transition zones.
    """
    
    eps = 1e-3
    
    pressures = []
    v_ranges = []
    
    for T in T_values:
        
        if T < 1.0:
        
            for p in p_values:
                
                roots = get_roots(p, T)
                
                if len(roots) == 3:
                    
                    v_range = [roots[0], roots[2]]
                    area = definite_integral(p, v_range, T)
                    
                    if abs(area) < eps:
                        
                        pressures.append(p)
                        v_ranges.append(v_range)
                        
                        break
                        
        elif T == 1.0:
            
            pressures.append(1.0)
            v_ranges.append([1.0])

    return pressures, v_ranges

In [9]:
def get_chemical_potential(p_values, v_values):
    """This function calculates chemical potential by integrating
    v(p) isotherms.\n

    Args:
        p_values: List of numpy arrays containing the pressures of
        the isotherms.\n
        v_values: List of numpy arrays containing the volumes of
        the isotherms.\n

        
    Returns:
        mu: List of numpy arrays containing the chemical potentials of
        the isotherms.
    """
    
    mu = []
    
    
    for i in range(len(v_values)):

        v = v_values[i]
        p = p_values[i]
        
        pot = [10.0] #starting random value
        
        l = np.size(p)
        
        for j in range(1, l):
            pot.append(pot[j-1] + v[j]*(p[j] - p[j-1]))
        
        mu.append(pot)
        
    return mu

In [10]:
def update_tracer(change):
    
    tracer_117_003.visible = True
    
    if v_slider.value == v_slider.options[-1]:
        v_slider.disabled = True
    
    i = change.owner.index
    
    v = theo_v_values_inverted[T_slider.index][i]
    p = theo_p_values_inverted[T_slider.index][i]
    
    #v = dense_v_values_inverted[T_slider.index][i]
    #p = dense_p_values_inverted[T_slider.index][i]

    tracer_117_003.x, tracer_117_003.y = [p], [v]

    area_117_003.x = np.append(area_117_003.x, p)
    area_117_003.y = np.append(area_117_003.y, v)
        
    lines_117_004.x = np.append(lines_117_004.x, p)
    lines_117_004.y = np.append(lines_117_004.y, mu[T_slider.index][i])

In [11]:
def restart_chemical_potential(a):
    
    lines_117_004.x, lines_117_004.y = [], []
    area_117_003.x, area_117_003.y = [], []
    
    p_slider.value = p_slider.min
    p_slider.disabled = False

In [12]:
def change_tenperature(change):

    #p_slider.max = len(mu[T_slider.index])-1
    #restart_chemical_potential(None)
    
    obj = change.owner
    
    i = obj.index
    
    opacities = [0.0 for t in obj.options]
    opacities[i] = 1.0
    
    lines_117_001.opacities = opacities
    lines_117_002.opacities = opacities    
    
    #lines_117_001.x, lines_117_001.y = v_values, p_values[T_slider.index]
    #lines_117_002.x, lines_117_002.y = p_values[T_slider.index], v_values
    ##
    ##tracer_117_003.x, tracer_117_003.y = [lines_117_002.x[0]], [lines_117_002.y[0]]
    ##
    ##axis_y_004.scale =  bqs.LinearScale(min = min(mu[T_slider.index]), max = max(mu[T_slider.index]))
    ##lines_117_004.scales = {'x': scale_x_004, 'y': axis_y_004.scale}
    ##
    ##
    ##axis_y_004.tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4)
    
    #dense_p_values = get_relative_isotherms(dense_v_values, [obj.value])[0]
    #
    #i_in_range = np.where(dense_p_values < 2.0)
#
    #dense_v_values_filtered = np.take(dense_v_values, i_in_range)[0]
    #dense_p_values_filtered = np.take(dense_p_values, i_in_range)[0]
    #
    #dense_v_values_inverted = np.flip(dense_v_values_filtered)
    #dense_p_values_inverted = np.flip(dense_p_values_filtered)
    #
    #dense_p_values_rounded = np.round(np.flip(dense_p_values_filtered), 3)
    #
    #p_slider.options, p_slider.value = dense_p_values_rounded, dense_p_values_rounded[0]
#
    #new_mu = get_chemical_potential(dense_p_values_inverted, dense_v_values_inverted)
    #print(len(mu))
    #for i in range(len(mu)):
    #    if i < len(new_mu):
    #        mu[i] = new_mu[i]
    #    else:
    #        try:
    #            mu.pop(i)
    #        except:
    #            print(i)

In [13]:
def show_all_potentials(change):
    
    max_limit = 0.0
    min_limit = 100.0
    
    for pot in mu:
        max_pot = max(pot)
        min_pot = min(pot)
        
        if max_pot > max_limit:
            max_limit = max_pot
            
        if min_pot < min_limit:
            min_limit = min_pot
            
    axis_y_004.scale =  bqs.LinearScale(min = min_limit, max = max_limit)

    lines_117_004.scales = {'x': scale_x_004, 'y': axis_y_004.scale}
    axis_y_004.tick_values = np.linspace(0.0, max_limit, 4)

   
    lines_117_004.x, lines_117_004.y = [p.tolist() for p in dense_p_values_inverted], [m.tolist() for m in mu]

In [14]:
def update_text(change):
    
    obj = change.owner
    i = obj.index
    
    p_text.value = '<p> {:.3f} </p>'.format(theo_p_values_inverted[T_slider.index][i])

In [15]:
def hex_to_rgb(number_hex):
    """This function converts a hexadecimal color to its rgb
    equivalent.
    
    Args:
        number_hex: String containing the hexadecimal representation
        of the color.\n
        
    Returns:
        number_rgb: Tuple consisted of the 3 numbers of the rgb
        representation of the color.\n
    """
    
    if '#' in number_hex:
        number_hex = number_hex[1:]
    
    number_rgb = (int(number_hex[0:2], 16), \
                  int(number_hex[2:4], 16), \
                  int(number_hex[4:], 16))
    
    return number_rgb

In [16]:
def rgb_to_hex(number_rgb):
    """This function converts a rgb color to its hexadecimal
    equivalent.
    
    Args:
        number_rgb: Tuple consisted of the 3 numbers of the rgb
        representation of the color.\n
        
    Returns:
        number_hex: String containing the hexadecimal representation
        of the color.\n
    """
    
    number_rgb = '#' \
    + format(number_rgb[0], '02x') \
    + format(number_rgb[1], '02x') \
    + format(number_rgb[2], '02x')
    
    return number_rgb

In [17]:
def generate_gradient(initial, final, length):
    """This function generates a color gradient consisted of N
    colors from the initial to the final.
    
    Args:
        initial: String of the hexadecimal representation of the
        initial color.\n
        final: String of the hexadecimal representation of the
        final color.\n
        length: Number of colors.\n
        
    Returns:
        colors: List consisted of strings of the hexadecimal
        colors.\n
    """
    
    i_r, i_g, i_b = hex_to_rgb(initial)
    f_r, f_g, f_b = hex_to_rgb(final)
    
    r_step = (f_r - i_r)/length
    g_step = (f_g - i_g)/length
    b_step = (f_b - i_b)/length
    
    r, g, b = i_r, i_g, i_b
    colors = []
    
    for i in range(length):

        h = rgb_to_hex((int(round(r)),int(round(g)),int(round(b))))
        
        colors.append(h)
        
        r = r + r_step
        g = g + g_step
        b = b + b_step

    return colors

In [18]:
v_values = np.logspace(
    start=np.log10([0.4])[0],
    stop=np.log10([5.2])[0],
    num=1000,
    endpoint=True
)
#v_values = np.logspace(-0.4, 0.7, 500)
T_values = np.round(np.linspace(0.85, 1.1, 15), 2)

if 1.0 not in T_values:
    T_values = np.sort(np.append(T_values, 1.0))

data = experimental_isotherms(
        p_range=[],
        v_range=v_values,
        T_range=T_values,
        fixed_T = True,
        fixed_p = False
)

expe_p_values = data[0]
theo_p_values = data[1]

p_limits = data[2]
v_limits = data[3]
T_limits = data[4]

colors = generate_gradient('#FF0000', '#FFfa00', len(T_limits)) 
opacities = [0.0 for t in T_values]
opacities[0] = 1.0

In [19]:
################################
######1ST BLOCK#################
################################

# This block shows the p(v,T) (fig_117_001) and
# v(p,T) (fig_117_002) figures.


block_1 = widgets.VBox(
    [],
    layout=widgets.Layout(
        align_items='center'
    )
)

scale_x = bqs.LinearScale(min = 0.4, max = 5.0)
scale_y = bqs.LinearScale(min = 0.0, max = 2.0)

axis_x_001 = bqa.Axis(
    scale=scale_x,
    tick_format='.2f',
    tick_style={'font-size': '15px'},
    tick_values = [1.0, 2.0, 3.0, 4.0, 5.0],
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    label='v',
    label_location='middle',
    label_style={'stroke': 'black', 'default-size': 35},
    label_offset='50px'
)

axis_y_001 = bqa.Axis(
    scale=scale_y,
    tick_format='.1f',
    tick_style={'font-size': '15px'},
    tick_values = [0, 1.0, 2.0],
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    orientation='vertical',
    label='p',
    label_location='middle',
    label_style={'stroke': 'red', 'default_size': 35},
    label_offset='50px'
)

fig_117_001 = Figure(
    title='p vs v (fixed T, reduced variables)',
    marks=[],
    axes=[axis_x_001, axis_y_001],
    animation_duration=250, 
    legend_location='top-right',
    background_style= {'fill': 'white',  'stroke': 'black'},
    min_aspect_ratio=1.0,
    max_aspect_ratio=1.0,
    fig_margin=dict(top=65, bottom=60, left=60, right=20),
    toolbar = True
)

lines_117_001 = bqm.Lines(
    x = v_values, 
    y = np.array(theo_p_values), 
    scales = {'x': scale_x, 'y': scale_y}, 
    opacities = opacities,
    visible = True,
    colors = colors,
)

fig_117_001.marks = [lines_117_001]

axis_x_002 = bqa.Axis(
    scale=scale_y,
    tick_format='.1f',
    tick_style={'font-size': '15px'},
    tick_values = [0, 1.0, 2.0],
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    label='p',
    label_location='middle',
    label_style={'stroke': 'black', 'default-size': 35},
    label_offset='50px'
)

axis_y_002 = bqa.Axis(
    scale=scale_x,
    tick_format='.2f',
    tick_style={'font-size': '15px'},
    tick_values = [1.0, 2.0, 3.0, 4.0, 5.0],
    grid_lines = 'none',
    grid_color = '#8e8e8e', 
    orientation='vertical',
    label='v',
    label_location='middle',
    label_style={'stroke': 'red', 'default_size': 35},
    label_offset='50px'
)

fig_117_002 = Figure(
    title='v vs p (fixed T, reduced variables)',
    marks=[],
    axes=[axis_x_002, axis_y_002],
    animation_duration=250,
    legend_location='top-right',
    background_style= {'fill': 'white',  'stroke': 'black'},
    min_aspect_ratio=1.0,
    max_aspect_ratio=1.0,
    fig_margin=dict(top=65, bottom=60, left=60, right=20),
    toolbar = True,
)

lines_117_002 = bqm.Lines(
    x = np.array(theo_p_values),
    y = np.array([v_values for p in theo_p_values]),   
    scales = {'x': scale_y, 'y': scale_x}, 
    opacities = opacities,
    visible = True,
    colors = colors
)

fig_117_002.marks = [lines_117_002]

block_1.children = [
    widgets.HBox([
        fig_117_001,
        fig_117_002
    ], 
        layout=widgets.Layout(width='100%')
    )
]

In [20]:
################################
######2ND BLOCK##############
################################

# This block shows the slider to control the tenperature 
# (T_slider).

block_2 = widgets.VBox(
    [],
    layout=widgets.Layout(align_items='center')
)

T_slider = widgets.SelectionSlider(
    options= T_values,
    value=T_values[0],
    description=r'\( T \)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(
        width = '300px',
        height = 'auto',
        margin='0 0 0 50px'
    )
)

T_slider.observe(change_tenperature, 'value')

block_2.children = [T_slider]

In [21]:
################################
######3RD BLOCK################
################################

# This block shows the v(p,T) (fig_117_003) and
# mu(p,T) (fig_117_004) figures.

block_3 = widgets.HBox(
    [],
    layout=widgets.Layout(align_items='center')
)

dense_v_values = np.linspace(min(v_values), max(v_values), 10000)
dense_p_values = get_relative_isotherms(dense_v_values, T_values)

dense_v_values_filtered = []#np.array([])
dense_p_values_filtered = []#np.array([])
#
dense_v_values_inverted = []#np.array([])
dense_p_values_inverted = []#np.array([])

dense_p_values_rounded = []#np.array([])

for i in range(len(T_values)):    

    i_in_range, = np.where(dense_p_values[i] < 2.0)

    dense_v_values_filtered.append(np.take(dense_v_values, i_in_range))
    dense_p_values_filtered.append(np.take(dense_p_values[i], i_in_range))
    
    dense_v_values_inverted.append(np.flip(dense_v_values_filtered[i]))
    dense_p_values_inverted.append(np.flip(dense_p_values_filtered[i]))
    
    dense_p_values_rounded.append(np.round(np.flip(dense_p_values_filtered[i]), 3))
    #dense_v_values_filtered = np.append(dense_v_values_filtered, np.take(dense_v_values, i_in_range))
    #dense_p_values_filtered = np.append(dense_p_values_filtered, np.take(dense_p_values[i], i_in_range))
#
    #dense_v_values_inverted = np.append(dense_v_values_inverted, np.flip(dense_v_values_filtered))
    #dense_p_values_inverted = np.append(dense_p_values_inverted, np.flip(dense_p_values_filtered[i]))
#
    #dense_p_values_rounded = np.append(dense_p_values_rounded, np.round(np.flip(dense_p_values_filtered[i]), 3))

fig_117_003 = Figure(
    title='v vs p (fixed T, reduced variables)',
    marks=[],
    axes=[axis_x_002, axis_y_002],
    animation_duration=0, #500,
    legend_location='top-right',
    background_style= {'fill': 'white',  'stroke': 'black'},
    min_aspect_ratio=1.0,
    max_aspect_ratio=1.0,
    fig_margin=dict(top=65, bottom=60, left=60, right=20),
    toolbar = True,
)

area_117_003 = bqm.Lines(
    x = [], 
    y = [], 
    scales = {'x': scale_y, 'y': scale_x}, 
    opacities = [1.0],
    visible = True, #True, #t == '1.00',
    colors = ['#39362d'],
    fill = 'bottom',
    fill_colors = ['#ffd429'],
    fill_opacities = [0.4]
    #labels = T_values_str,
)

tracer_117_003 = bqm.Scatter(
    name = '',
    x = [0.0],
    y = [0.0],
    scales = {'x': scale_y, 'y': scale_x}, 
    opacity = [1.0, 0.0],
    visible = False,
    colors = ['#2807a3'],
)

fig_117_003.marks = [
    lines_117_002,
    area_117_003,
    tracer_117_003
]

theo_p_values_inverted = [np.flip(p_values) for p_values in theo_p_values]
theo_v_values_inverted = [np.flip(v_values) for p_values in theo_p_values]

#mu = get_chemical_potential(dense_p_values_inverted, dense_v_values_inverted)
mu = get_chemical_potential(
    theo_p_values_inverted,
    theo_v_values_inverted
)

for i in range(len(mu)):
    mu[i] = np.array(mu[i]) - 10.0*T_slider.options[i]

p_text = widgets.HTML(
    value="<p>" + str(dense_p_values_rounded[T_slider.index][i]) + "</p>",
    layout = widgets.Layout(height='auto', margin='10px 0 0 10px')
)

v_slider = widgets.SelectionSlider(
    options= theo_v_values_inverted[0],
    value=theo_v_values_inverted[0][0],
    description=r'\( v \)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout = widgets.Layout(
        width = '60%',
        height = 'auto',
        margin='0 0 0 50px'
    )
)

v_slider.observe(update_tracer, 'value')
v_slider.observe(update_text, 'value')
                 
#p_slider = widgets.IntSlider(
#    min=0,
#    max=len(mu[T_slider.index])-1,
#    value=0,
#    description=r'\( v \)',
#    disabled=False,
#    continuous_update=True,
#    orientation='horizontal',
#    readout=False,
#    layout = widgets.Layout(width = '60%', height='auto', margin='0 0 0 50px')
#)
#
#p_slider.observe(update_tracer, 'value')
#p_slider.observe(update_text, 'value')


#play = widgets.Play(
#    interval=1,
#    value=0,
#    min=0,
#    max=len(p_slider.options),
#    step=1,
#    description="Press play",
#    disabled=False
#)
#widgets.jslink((play, 'value'), (p_slider, 'index'));



scale_x_004 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_004 = bqs.LinearScale(min = min(mu[T_slider.index]), max = max(mu[T_slider.index]))

axis_x_004 = bqa.Axis(scale=scale_x_004,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(min(theo_p_values_inverted[T_slider.index]), max(theo_p_values_inverted[T_slider.index]), 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_004 = bqa.Axis(
                scale=scale_y_004,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 0,
                #tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='mu',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_117_004 = Figure(
      title='mu vs p (fixed T)',
       marks=[],
       axes=[axis_x_004, axis_y_004],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=65, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

lines_117_004 = bqm.Lines(
       x = [], 
       y = [], 
       scales = {'x': scale_x_004, 'y': scale_y_004}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = colors,
       #labels = T_values_str,
       )

fig_117_004.marks = [lines_117_004]

restart_button = widgets.Button(
        description='Clean figure',
        disabled=False,
        button_style='', 
        tooltip="",
        layout = widgets.Layout(height='auto')
    )

restart_button.on_click(restart_chemical_potential)


show_all_button = widgets.Button(
        description='Show all potentials',
        disabled=False,
        button_style='',
        tooltip="",
        layout = widgets.Layout(height='auto')
    )

show_all_button.on_click(show_all_potentials)

block_3.children = [widgets.VBox([fig_117_003, widgets.HBox([v_slider, p_text])], layout=widgets.Layout(height='auto')),
                         widgets.VBox([fig_117_004, restart_button, show_all_button], layout=widgets.Layout(align_items='center'))]

In [22]:
block_3

################################
######4TH BLOCK#################
################################

block_4 = widgets.HBox([], layout=widgets.Layout(align_items='center'))

trans_pressures = []
trans_pressures_repeated = []
trans_volumens = []
trans_potentials = []

for i in range(len(T_values)):
    T = T_values[i]
    p = p_values[i]
    
    if T < 1.0:
        p_trans, v_range = find_real_fixed_T(p, T)

        if p_trans is not None:
            trans_pressures = np.append(trans_pressures, p_trans)
            trans_pressures_repeated = np.append(trans_pressures_repeated, [p_trans, p_trans])
            trans_volumens = np.append(trans_volumens, v_range)
            
        p_v = dense_p_values_inverted[i]
        index, = np.where(abs(p_v - p_trans) < 1e-4)
        if len(index) != 0:
            trans_potentials = np.append(trans_potentials, mu[i][index[0]])
                
    elif T == 1.0:
        
            trans_pressures = np.append(trans_pressures, [1.0])
            trans_pressures_repeated = np.append(trans_pressures_repeated, [1.0])
            trans_volumens = np.append(trans_volumens, 1.0)
            
            p_v = dense_p_values_inverted[i]
            index, = np.where(abs(p_v - 1.0) < 1e-10)
            if len(index) != 0:
                trans_potentials = np.append(trans_potentials, mu[i][index[0]])
            
lines_117_005 = bqm.Lines(
       x = v_values,
       y = p_values, 
       scales = {'x': scale_x, 'y': scale_y}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = gradient,
       #labels = T_values_str,
       )

scatter_117_005 = bqm.Scatter(
    name = '',
    x = trans_volumens,
    y = trans_pressures_repeated,
    scales = {'x': scale_x, 'y': scale_y}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    default_size = 15
)   

fig_117_005 = Figure(
      title='v vs p (fixed T, reduced variables)',
       marks=[lines_117_005, scatter_117_005],
       axes=[axis_x_001, axis_y_001],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=65, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )



max_limit = 0.0
min_limit = 100.0

for pot in mu:
    max_pot = max(pot)
    min_pot = min(pot)

    if max_pot > max_limit:
        max_limit = max_pot

    if min_pot < min_limit:
        min_limit = min_pot
      
        
scale_x_006 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_006 = bqs.LinearScale(min = min_limit, max = max_limit)

axis_x_006 = bqa.Axis(scale=scale_x_006,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(0.0, 2.0, 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_006 = bqa.Axis(
                scale=scale_y_006,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks = 0,
                #tick_values = np.linspace(min(mu[T_slider.index]), max(mu[T_slider.index]), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='mu',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

lines_117_006 = bqm.Lines(
       x = [p.tolist() for p in dense_p_values_inverted],
       y = [m.tolist() for m in mu], 
       scales = {'x': scale_x_006, 'y': scale_y_006}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = gradient,
       #labels = T_values_str,
       )

scatter_117_006 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = trans_potentials,
    scales = {'x': scale_x_006, 'y': scale_y_006}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    default_size = 15
) 

fig_117_006 = Figure(
      title='mu vs p (fixed T)',
       marks=[lines_117_006, scatter_117_006],
       axes=[axis_x_006, axis_y_006],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=65, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

block_4.children = [fig_117_005, fig_117_006]

################################
######5TH BLOCK#################
################################

block_5 = widgets.HBox([], layout=widgets.Layout(align_items='center'))

scale_y_008 = bqs.LinearScale(min = min(T_values), max = max(T_values), reverse = True)
      
axis_x_008 = bqa.Axis(scale=scale_x_006,
                    tick_format='.2f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    tick_values = np.linspace(0.0, 2.0, 5),
                    #num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='p',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='30px',
                    side='top')    

axis_y_008 = bqa.Axis(
                scale=scale_y_008,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(min(T_values), max(T_values), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px',
                )

lines_117_008 = bqm.Lines(
       x = trans_pressures,
       y = T_values, 
       scales = {'x': scale_x_006, 'y': scale_y_008}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_008 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = T_values,
    scales = {'x': scale_x_006, 'y': scale_y_008}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    default_size = 15
)   

fig_117_008 = Figure(
      title='T vs p (fixed v, reduced variables)',
       marks=[lines_117_008, scatter_117_008],
       axes=[axis_x_008, axis_y_008],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=120, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

scale_y_007 = bqs.LinearScale(min = min(T_values), max = max(T_values))
          
axis_y_007 = bqa.Axis(
                scale=scale_y_007,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(min(T_values), max(T_values), 4),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='T',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px',
                )

lines_117_007 = bqm.Lines(
       x = trans_pressures,
       y = T_values, 
       scales = {'x': scale_x_006, 'y': scale_y_007}, 
       opacities = [1.0],
       visible = True, #True, #t == '1.00',
       colors = ['red'],
       #labels = T_values_str,
       )

scatter_117_007 = bqm.Scatter(
    name = '',
    x = trans_pressures,
    y = T_values,
    scales = {'x': scale_x_006, 'y': scale_y_007}, 
    #opacities = [1.0],
    visible = True,
    colors = ['black'],
    default_size = 15
)   

fig_117_007 = Figure(
      title='T vs p (fixed v, reduced variables)',
       marks=[lines_117_007, scatter_117_007],
       axes=[axis_x_006, axis_y_007],
       animation_duration=0, #500,
       #layout = widgets.Layout(align_self='center', width='75%'),
       legend_location='top-right',
       background_style= {'fill': 'white',  'stroke': 'black'},
       min_aspect_ratio=1.0,
       max_aspect_ratio=1.0,
       fig_margin=dict(top=50, bottom=60, left=60, right=20),
       toolbar = True,
       #layout = widgets.Layout(width='90%', height='300px')
      )

block_5.children = [fig_117_007, fig_117_008]

In [23]:
###################################
#############MAIN BLOCK############
###################################

main_block = widgets.VBox([], layout=widgets.Layout(align_items='center'))
main_block.children = [block_1,
                       block_2,
                       #block_3,
                       #block_4,
                       #block_5
                      ]

main_block